In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from pprint import pprint
import random
import tensorflow as tf
from tensorflow import keras
import math
import tensorflow_hub as hub

In [2]:
LABEL_INT_DICT = np.unique(pd.read_csv('ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = { k:v  for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_STR_DICT)

{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}


In [4]:
model = keras.models.load_model(
    'ucf11_weights/17-0.96.hdf5',
    custom_objects={'KerasLayer':hub.KerasLayer}
)

model.build([None, 256, 256, 3])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              5919312   
                                                                 
 dense_2 (Dense)             (None, 11)                14091     
                                                                 
Total params: 5,933,403
Trainable params: 5,872,795
Non-trainable params: 60,608
_________________________________________________________________


In [10]:
csv_path = 'ucf11_valid_vid.csv'
valid_df = pd.read_csv(csv_path)

idx = 21
elem = valid_df.iloc[idx]

cap = cv2.VideoCapture(
    elem['file_path']
)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter('movie.avi', fourcc, 30, (256, 256), True)

queue = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ = frame_ / 255.

    pred = model.predict(frame_[np.newaxis, ...]) #영상이랑 차원을 맞춰주기 위해 새로운 차원을 넣어줌
    queue.append(pred)
    
    results = np.array(queue).mean(axis=0)
    label = LABEL_INT_DICT[np.argmax(results)]
    
    frame = cv2.resize(frame, (256, 256))
    cv2.putText(frame, label, (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 5)
    writer.write(frame)

cap.release()
writer.release()

1/1 [==============================] - 0s 25ms/step


In [9]:
result = 11
print(np.argmax(result))

0
